<a href="https://colab.research.google.com/github/FabiolaAJ/chatbot-assistente-de-vagas/blob/main/Chatbot_Assistente_de_Vagas_de_emprego.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip -q install google-genai

In [ ]:
!pip install -q google-adk

In [ ]:
import os
from google.colab import userdata
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings
from IPython.display import clear_output

os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

warnings.filterwarnings("ignore")

# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
##########################################
# --- Agente 1: Buscador de Vagas --- #
##########################################
def agente_buscador_vagas_de_emprego(profissao, area, senioridade, data_de_hoje):

    buscador = Agent(
        name="agente_buscador_de_vagas",
        model="gemini-2.0-flash",
        instruction="""
        Você é um assistente simpático especializado na busca de vagas de emprego.
        Use a ferramenta de busca do google (google_search) para encontrar até 5 vagas relevantes e atuais para a profissão e senioridade do usuário.

          1. **Interpretação:** Analise profissão e senioridade. Se a senioridade faltar, pergunte.
          2. **Busca (google_search):** Combine profissão, área, senioridade e informe se a vaga é hibrida, presencial ou remoto. Foque em resultados que provavelmente contenham links diretos para a página da vaga.
          3. **Filtragem:** Priorize vagas recentes verificando a data de hoje (data_de_hoje). Descarte vagas antigas ou encerradas.
          4. **Resposta (máximo 5 vagas):** Para cada vaga, retorne título, empresa, localização (se disponível), breve descrição com requisitos para vaga, não faça questionamentos adicionais para o usuário.
          5. **Link:** [**Link direto para a página da vaga**. Este é um campo obrigatório. Tente ao máximo encontrar o link nos resultados da busca.]
          6. **Sem Resultados:** Se não houver vagas relevantes e atuais, informe o usuário.

          Priorize informações precisas e atuais.

        """,
        description="Agente que busca informações de vagas de empregos no Google",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Profissão: {profissao} \n Area:{area} \n Senioridade:{senioridade} \n Data de hoje: {data_de_hoje}"

    lancamentos = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos

In [ ]:
################################################
# --- Agente 2: Planejador de carreira e recrutamento --- #
################################################
def agente_planejador_de_carreira(profissao, area, senioridade, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Você é um especialista em carreira e recrutamento muito gentil e simpático, com vasta experiência em ajudar candidatos a se prepararem para processos seletivos.
        Gere dicas para se destacar, considerando os seguintes tópicos:
             1. Quais habilidades e experiências o candidato deve enfatizar?
             2. Forneça até 2 dicas concisas sobre como o usuário pode se organizar para se candidatar as vagas (ex: adaptar currículo, pesquisar a empresa).
             3. Ao final cite dicas gerais de como se destacar no mercado de trabalho de acordo com a profissão do usuário
        """,
        description="Agente planejador especialista em carreira",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Profissão: {profissao} \n Area:{area}  \n Senioridade:{senioridade} \n Buscados: {lancamentos_buscados}"

    # Executa o agente
    lancamento_planejamento = call_agent(planejador, entrada_do_agente_planejador)
    return lancamento_planejamento

In [20]:
def agente_responder_duvidas(profissao, area, senioridade, duvida, lancamentos_buscados, lancamento_planejamento):

    buscador = Agent(
        name="agente_buscador_de_vagas",
        model="gemini-2.0-flash",
        instruction="""
        Você é um assistente de carreira e você sempre responde de forma gentil e de facil entendimento.
        Use a ferramenta de busca do google (google_search) para tirar a dúvida do usuário sobre sua carreira, responda de forma resumida.
        """,
        description="Agente para tirar dúvidas de carreira",
        tools=[google_search]
    )

    entrada_do_agente_responder_duvidas = f"Dúvida: {duvida}\n Profissao:{profissao} \n Area:{area} \n Senioridade:{senioridade} \n Buscados: {lancamentos_buscados} \n Planejador: {lancamento_planejamento}"

    lancamentos = call_agent(buscador, entrada_do_agente_responder_duvidas)
    return lancamentos

In [ ]:
def obter_input_obrigatorio(mensagem_inicial, mensagem_erro):
    # Exibe a mensagem inicial para o usuário, solicitando a entrada.
    valor = input(f"\n{mensagem_inicial} ")

    # Inicia um loop 'while' que continuará executando enquanto a condição for verdadeira.
    # A condição 'not valor.strip()' verifica se a variável 'valor' está vazia ou contém apenas espaços em branco.
    # 'valor.strip()' remove os espaços em branco do início e do final da string.
    # Se após a remoção dos espaços, a string estiver vazia, a condição será 'True' e o loop continuará.
    while not valor.strip():
        # Exibe a mensagem de erro informando ao usuário que a entrada é obrigatória.
        print(f"{mensagem_erro}")

        # Solicita novamente a entrada do usuário, exibindo a mensagem inicial novamente.
        valor = input(f"\n{mensagem_inicial}")

    # Quando o loop 'while' terminar (ou seja, o usuário digitou algo que não é apenas espaço),
    # a função retorna o valor digitado, com quaisquer espaços em branco iniciais e finais removidos
    # usando o método 'strip()'.
    return valor.strip()

In [ ]:
def tirar_duvidas(profissao, area, senioridade, resposta_buscador, resposta_planejador):
    questionario = "S"

    while questionario.upper() == "S" :
      print("\n Ficou com alguma dúvida? Pode contar comigo!😉 (caso não tenha dúvidas digite 'N') \n")
      duvida = input("")
      if(duvida.upper() == 'N'):
        break;
      else:
        print("\n🔄 Processando resposta...")
        resposta_duvida = agente_responder_duvidas(profissao, area, senioridade, duvida, resposta_buscador, resposta_planejador)
        display(to_markdown(resposta_duvida))

        print("\n Se deseja tirar outra dúvida digite 'S'(caso não tenha, digite qualquer tecla): \n")
        questionario = input("")

        if(questionario.upper() != "S"):
          break;


In [ ]:
def exibir_resultados(profissao, area, senioridade):
    print(f"")

    data_de_hoje = date.today().strftime("%d/%m/%Y")

    resposta_buscador = agente_buscador_vagas_de_emprego(profissao, area, senioridade, data_de_hoje)
    print("\n 📝 Essas foram as vagas encontradas! \n")
    display(to_markdown(resposta_buscador))
    print("--------------------------------------------------------------")

    print("\n🔄 Processando resultados de planejamento...")
    resposta_planejador = agente_planejador_de_carreira(profissao, area, senioridade, resposta_buscador)
    print("\n 🚀 Algumas dicas para decolar em seu processo seletivo! 🚀 \n")
    display(to_markdown(resposta_planejador))
    print("--------------------------------------------------------------")

    tirar_duvidas(profissao, area, senioridade, resposta_buscador, resposta_planejador)

    return

In [23]:
def main():
  exibir_questionario = "1";

  while exibir_questionario == "1" :
    clear_output(wait=True)

    print("\n Olá, tudo bem por aí? 😊 Que bom te ter aqui!")
    print("\n Sou seu assistente virtual de carreira, pronto para te dar uma mãozinha especial na busca daquela vaga de emprego e te preparar para o sucesso nas entrevistas! 🎉")
    print("\n--------------------------------------------------------------")

    profissao = obter_input_obrigatorio("❓Me conta qual a profissão que te interessa encontrar vagas hoje? ", "/n Hmm, para que eu te ajude a achar as melhores vagas, preciso que me conte qual profissão você busca, tá bem? 😉 ")
    area = obter_input_obrigatorio("❓Legal! qual a área específica dentro dessa profissão você deseja atuar? (Por exemplo: UX, Gestão, Frontend, Backend) " , "É importante me informar qual área específica dentro dessa profissão deseja atuar!😊 ")
    senioridade = obter_input_obrigatorio("❓E qual o seu nível de experiência? (Por exemplo: pleno, sênior, júnior, estágio...) ", "Porfavor, digite seu nível de experiência para refinar a busca! 😊 ")

    print("\n🔄 Processando resultados...")
    exibir_resultados(profissao, area, senioridade)

    print("\n Se deseja procurar outra vaga, digite 1 (para encerrar digite qualquer tecla): \n")

    exibir_questionario = input("")

    if(exibir_questionario != '1'):
      print("\n Até a próxima! 😊")
      break;


main()




 Olá, tudo bem por aí? 😊 Que bom te ter aqui!

 Sou seu assistente virtual de carreira, pronto para te dar uma mãozinha especial na busca daquela vaga de emprego e te preparar para o sucesso nas entrevistas! 🎉

--------------------------------------------------------------

❓Me conta qual a profissão que te interessa encontrar vagas hoje?  Programador

❓Legal! qual a área específica dentro dessa profissão você deseja atuar? (Por exemplo: UX, Gestão, Frontend, Backend)  Fullstack

❓E qual o seu nível de experiência? (Por exemplo: pleno, sênior, júnior, estágio...)  Júnior

🔄 Processando resultados...


 📝 Essas foram as vagas encontradas! 



> Com base nas buscas realizadas, encontrei algumas vagas que correspondem ao seu perfil. Aqui estão algumas opções:
> 
> 1.  **Desenvolvedor Full-Stack Júnior - PHP+React**
> 
>     *   **Empresa:** SENAI/SC - Soluções Digitais
>     *   **Localização:** Teletrabalho
>     *   **Descrição:** Atuar no desenvolvimento e manutenção de sistemas web voltados para a área educacional, utilizando PHP (Symfony) no backend e React no frontend. Requisitos incluem ensino médio completo com graduação em andamento ou técnico em áreas de TI, e 6 meses de experiência em desenvolvimento de softwares, análise e manutenção de dados em banco de dados, bibliotecas de front-end, ferramentas de versionamento, PHP com Framework Symfony, Testes automatizados com PHPUnit, React com componentização e estilização com Tailwind e/ou Styled Components.
>     *   **Link:** [Link para a vaga](https://www.infojobs.com.br/vaga-de-01245-2025-desenvolvedor-full-stack-junior-php-react-em-todo-brasil__6359685.aspx)
> 2.  **Desenvolvedor Full Stack Junior**
> 
>     *   **Empresa:** Grupo Roma Brasil
>     *   **Local de trabalho:** Belo Horizonte - MG
>     *   **Modelo de trabalho:** Presencial
>     *   **Descrição:** Documentar as etapas de desenvolvimento, executar e documentar testes de alta complexidade, e desenvolver e manter softwares de acordo com as especificações. Requisitos incluem ensino superior completo ou cursando em Ciências da Computação ou Sistemas de Informação, experiência em desenvolvimento de Software sob medida, conhecimentos sólidos em OOP, MVC e Integrações Rest, conhecimentos sólidos na plataforma Microsoft Azure, e domínio de linguagens de programação (Frameworks, PHP, MYSQL, SQL Server, Laravel, API).
>     *   **Link:** [Link para a vaga](https://www.gupy.us/offers/2598148?utm_campaign=google_jobs&utm_source=google_jobs&utm_medium=organic)
> 3.  **Desenvolvedor(a) Júnior**
> 
>     *   **Empresa:** Escritório Inteligente
>     *   **Localização:** Jundiaí, São Paulo, Brasil
>     *   **Descrição:** Vaga para Desenvolvedor(a) Júnior.
>     *   **Link:** Ver vaga no LinkedIn.
> 4.  **Desenvolvedor Fullstack JR**
> 
>     *   **Empresa:** HUB Floripa Participações LTDA
>     *   **Localização:** Florianópolis, SC
>     *   **Descrição:** O Hub de tecnologia e inovação está construindo um time de alta performance e busca um Desenvolvedor Fullstack JR.
>     *   **Link:** Não foi possivel encontrar o link direto para a vaga.
> 5.  **Programador Full Stack Junior [BH]**
> 
>     *   **Empresa:** Teknisa
>     *   **Local de trabalho:** Belo Horizonte - MG
>     *   **Modelo de trabalho:** Híbrido
>     *   **Descrição:** Participar de todas as etapas de um processo de desenvolvimento de software, desde a concepção até a implementação e manutenção, compartilhar conhecimentos com a equipe, considerar escalabilidade, performance e qualidade de código, sugerir melhorias no processo de desenvolvimento e buscar aperfeiçoamento tecnológico.
>     *   **Link:** [Link para a vaga](https://www.gupy.us/offers/2519243?utm_campaign=google_jobs&utm_source=google_jobs&utm_medium=organic)
> 


--------------------------------------------------------------

🔄 Processando resultados de planejamento...

 🚀 Algumas dicas para decolar em seu processo seletivo! 🚀 



> Olá! Com prazer, vou te ajudar a se destacar nessas oportunidades de Fullstack Júnior.
> 
> **1. Habilidades e Experiências a Enfatizar:**
> 
> *   **Domínio do Stack:** Destaque seu conhecimento em linguagens como PHP e frameworks como Symfony (mencionado na vaga 1), além de React para o frontend. Se você tiver experiência com outras tecnologias comuns em fullstack, como Node.js, Python ou Angular, também vale a pena mencionar.
> *   **Banco de Dados:** Mencione sua experiência com bancos de dados relacionais (MySQL, SQL Server) e, se tiver, NoSQL.
> *   **Metodologias Ágeis:** Experiência com metodologias ágeis (Scrum, Kanban) é um diferencial, pois muitas empresas as utilizam.
> *   **Soft Skills:** Habilidades como trabalho em equipe, comunicação e resolução de problemas são cruciais. Demonstre como você colaborou em projetos e lidou com desafios.
> *   **Cloud:** Se tiver experiência com plataformas como Microsoft Azure (vaga 2) ou AWS, mencione isso.
> 
> **2. Dicas de Organização para Candidaturas:**
> 
> *   **Adapte seu currículo:** Para cada vaga, revise seu currículo e destaque as habilidades e experiências mais relevantes para aquela posição específica. Use palavras-chave presentes na descrição da vaga.
> *   **Pesquise a empresa:** Antes da entrevista, pesquise a fundo a empresa. Entenda seus produtos, serviços, cultura e desafios. Isso demonstra interesse e te ajuda a formular perguntas relevantes.
> 
> **3. Dicas Gerais para se Destacar no Mercado de Trabalho como Programador Fullstack:**
> 
> *   **Mantenha-se atualizado:** A área de tecnologia está em constante evolução. Invista em cursos, workshops e leitura de artigos para se manter atualizado com as últimas tendências e tecnologias.
> *   **Crie um portfólio:** Desenvolva projetos pessoais e contribua para projetos open source. Isso demonstra suas habilidades de forma prática e impressiona recrutadores.
> *   **Networking:** Participe de eventos da área, grupos de discussão online e conecte-se com outros profissionais no LinkedIn. O networking pode abrir portas para oportunidades de emprego e parcerias.
> *   **Inglês:** A fluência em inglês é fundamental, pois muitos materiais de estudo, documentações e empresas são internacionais.
> 
> Espero que estas dicas te ajudem a brilhar no processo seletivo e a conquistar a vaga dos seus sonhos!


--------------------------------------------------------------

 Ficou com alguma dúvida? Pode contar comigo!😉 (caso não tenha dúvidas digite 'N') 

Como me destacar no mercado?

🔄 Processando resposta...


> Para se destacar no mercado como programador Fullstack Júnior, foque em aprimorar e demonstrar suas habilidades técnicas, como o domínio de PHP, React e bancos de dados. Adapte seu currículo para cada vaga, destacando as habilidades mais relevantes e utilizando palavras-chave presentes na descrição da vaga. Além disso, mantenha-se atualizado com as últimas tendências tecnológicas, construa um portfólio com projetos pessoais e invista em networking para ampliar suas oportunidades.



 Se deseja tirar outra dúvida digite 'S'(caso não tenha, digite qualquer tecla): 

N

 Se deseja procurar outra vaga, digite 1 (para encerrar digite qualquer tecla): 

0

 Até a próxima! 😊
